In [2]:
%pip install transformers
%pip install setfit
%pip install pyarrow

Note: you may need to restart the kernel to use updated packages.
^C
Traceback (most recent call last):
  File "/usr/lib/python3.8/runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.8/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/home/blake/venvs/fyp/lib/python3.8/site-packages/pip/__main__.py", line 16, in <module>
    from pip._internal.cli.main import main as _main  # isort:skip # noqa
  File "/home/blake/venvs/fyp/lib/python3.8/site-packages/pip/_internal/cli/main.py", line 10, in <module>
    from pip._internal.cli.autocompletion import autocomplete
  File "/home/blake/venvs/fyp/lib/python3.8/site-packages/pip/_internal/cli/autocompletion.py", line 9, in <module>
    from pip._internal.cli.main_parser import create_main_parser
  File "/home/blake/venvs/fyp/lib/python3.8/site-packages/pip/_internal/cli/main_parser.py", line 7, in <module>
    from pip._internal.cli import cmdoptions
  File "/ho

In [9]:
def train_few_shot_model(pretrainied_model_string, train_path, test_path, num_epoch, num_iteration):
  from setfit import SetFitModel, SetFitTrainer
  import pandas as pd
  import pyarrow as pa
  import datasets

  train_dataframe = pd.read_csv(train_path).sample(frac=1)
  test_dataframe = pd.read_csv(test_path).sample(frac=1)

  train_dataset = datasets.Dataset(pa.Table.from_pandas(train_dataframe))
  test_dataset = datasets.Dataset(pa.Table.from_pandas(test_dataframe))

  model = SetFitModel.from_pretrained(pretrainied_model_string)

  trainer = SetFitTrainer(
    model=model,
    train_dataset=train_dataset,
    metric="accuracy",
    batch_size=16,
    num_iterations=num_iteration, # The number of text pairs to generate for contrastive learning
    num_epochs=num_epoch, # The number of epochs to use for contrastive learning
    column_mapping={"text": "text", "class": "label"} # Map dataset columns to text/label expected by trainer
  )

  trainer.train()

  # output filename
  modelnamesplit = pretrainied_model_string.split('/')
  modelname = modelnamesplit[len(modelnamesplit)-1]
  trainnamesplit = train_path.split('/')
  trainname = trainnamesplit[len(trainnamesplit)-1]
  output_filename = modelname+"__"+trainname+"__"+str(num_epoch)+"epochs"
  
  test_dataframe['predicted'] = trainer.model.predict(test_dataframe['text'])
  test_dataframe.to_csv(output_filename, index=False)


In [10]:
epochs = 1
modelnames = [
  "facebook/bart-large-mnli",
  "morit/french_xlm_xnli"
]
# cheeky for testing
modelnames = ["prajjwal1/bert-tiny-mnli"]

for modelname in modelnames:
  train_few_shot_model(
    "",
    "../data_ready/few_shot/train_pairs_random_4.csv",
    "../data_ready/unused_pairs_for_test_data.csv",
    epochs,
    20 #8x20x2 = 320
  )

No sentence-transformers model found with name /home/blake/.cache/torch/sentence_transformers/prajjwal1_bert-tiny-mnli. Creating a new one with MEAN pooling.
Some weights of the model checkpoint at /home/blake/.cache/torch/sentence_transformers/prajjwal1_bert-tiny-mnli were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
Appl

'\nmodel names:\n  facebook/bart-large-mnli\n  morit/french_xlm_xnli\n\n'